# Utilizando diferentes atributos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

%matplotlib inline

# Base Info:
Nome: Rice Dataset (Commeo and Osmancik)     
Resumo: Um total de 3810 imagens de grãos de arroz foram tiradas de duas espécies (Cammeo e Osmancik).      
- As imagens foram pré-processadas e certos atributos foram inferidos     
- Sete atributos morfológicos foram obtidos para cada imagem de grão de arroz
    - Area (integer)
    - Perimeter (float)
    - Major_Axis_Length (float)
    - Minor_Axis_Length (float)
    - Eccentricity (float)
    - Extent (float)
- Classes:
    - Cammeo
    - Osmancik

	



In [2]:
df = pd.read_csv('rice.csv', header=None)

y = df.iloc[0:, 7].values

# Esse trecho de código é uma tentativa de evitar o overflow 
# quando se usa determinados parâmetros
# Ainda não consegui resolver o problema

# X = df.iloc[0:, [0,1,2,3,4,5,6]].values

# for i, label in enumerate(X):
#     print(i)
#     print(X[i])
#     label[0] = label[0] / 10000
#     label[1] = label[1] / 100
#     label[2] = label[2] / 100
#     label[3] = label[3] / 10
#     label[5] = label[5] / 10000
#     print(X[i])

In [3]:
y = np.where(y == 'Cammeo', -1, 1) # realizado o tratamento do classe categorica (string -> int)
y

array([-1, -1, -1, ...,  1,  1,  1])

# Objeto Neuronio, seus atributos e métodos

In [4]:
# base
# classe Neuronio e aprendizado estocastico/online

class Neuronio(object):

    def __init__(self, eta=0.1, epocas=50, fativ='perceptron',showErro=False):
        self.eta = eta
        self.epocas = epocas
        self.w_ = np.random.rand(1 + X.shape[1]) - 0.5
        self.fativ = fativ
        self.showErro = showErro
        self.erro_ep = 0
        self.SqError_ = []

    def somat(self, X):
        return self.w_[0] + np.dot(X, self.w_[1:])

    def sinal(self, x):
        return np.where(x >= 0.0, 1, -1)

    def tanh(self, x):
      return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def d_tanh(self, x):
      return 1 - self.tanh(x) * self.tanh(x)

    def ReLU(self, x):
      return np.where(x > 0.0, x, 0.1*x)  #max(0, x) .... float

    def d_ReLU(self, x):
      return np.where(x > 0.0, 1, 0.1)

    def predict(self,X):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
#        print(f"sinalPredict={self.sinal(self.somat(X))}")  # debug
        return self.sinal(self.somat(X))
      elif self.fativ == 'linear':
        return self.somat(X)
      elif self.fativ == 'tanh':
        return self.tanh(self.somat(X))
      elif self.fativ == 'ReLU':
        return self.ReLU(self.somat(X))
      else:
        return 11

    def deltaW(self, erro):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
        atualiza = self.eta * erro
      elif self.fativ == 'linear':
        atualiza = self.eta * erro
      elif self.fativ == 'tanh':
#        print(f"> eta={self.eta} tanh={self.tanh(erro)} dtanh={self.d_tanh(erro)} erro={erro}")
        atualiza = self.eta * self.d_tanh(erro) * erro
      elif self.fativ == 'ReLU':
        atualiza = self.eta * self.d_ReLU(erro) * erro
      else:
        print(f"Função de ativação '{self.fativ}' desconhecida")
        exit()
      return atualiza

    def treinaGD(self,X,y):
      self.SqError_ = []
      self.erros_classif_ = []
      for ep in range(self.epocas):
        erro_classif = 0
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        Xs = X[indices]
        ys = y[indices]
        outputs = self.predict(Xs)
        erros = (ys - outputs)
       # print(outputs,erros)
        erro_classif = np.where(ys*outputs < 0, 1, 0).sum()
        self.erros_classif_.append(erro_classif)
        if ep < (self.epocas-1):
          self.w_[1:] += self.eta * Xs.T.dot(erros)
          self.w_[0] += self.eta * erros.sum()
        SqError = (erros**2).sum() / 2.0 # ou np.square(erros)/2
        self.SqError_.append(SqError)
        if self.showErro:
          if ep == 0:
            print(f"{'Época':^10}\tErro")
          else:
            print(f"{str(ep):^10}\t{erro_classif}")
      return self

    def treinaSGD(self, X, y):
        self.erros_ = []
        self.erros_classif_ = []
        self.SqError_ = []
        for ep in range(self.epocas):
            erro_ep = 0
            erro_classif = 0
            SqError = 0

            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            Xs = X[indices]
            ys = y[indices]

            for xi, target in zip(Xs, ys):
              output = self.predict(xi)
              erro = target - output
              SqError += erro*erro
              erro_ep += erro
              if (float(target)*float(output)) < 0:
                #print(f"ep={ep} {xi} target={target} output={output} t*o={target*output}") # debug
                erro_classif += 1

              if (ep < (self.epocas-1)):  #
                atualizacao = self.deltaW(erro)
                self.w_[0] +=  atualizacao * 1
                self.w_[1:] +=  atualizacao * xi

            self.erros_.append(erro_ep)
            self.SqError_.append(SqError)
            self.erros_classif_.append(erro_classif)
            if self.showErro:
              if ep == 0:
                print(f"{'Época':^10}\tErro")
              else:
                print(f"{str(ep):^10}\t{erro_classif}")

        return self

In [12]:
classifiers = []
W0_ = []
atributes_selected = [[2,3], [2,3,4], [2,3,4,6]]
# atributes_selected = [[1,2,3,4,5,6], [2,3]]

for atributes in atributes_selected:
    print("Atributos selecionados" + str(atributes))

    X = df.iloc[0:, atributes].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=102)

    np.random.seed(35)
    classifier = Neuronio(fativ='tanh', eta=0.0001, epocas=300, showErro=False)
    W0 = classifier.w_.copy()
    W0_.append(W0)
    classifier.treinaSGD(X_train, y_train)
    classifiers.append(classifier)

    pred = classifier.predict(X_test)
    PercClassifica = (100 * np.where(pred*y_test < 0, 0, 1).sum() / len(y_test))
    print("Porcentagem de acertos: " + str(round(PercClassifica,2)) + "%")




Atributos selecionados[2, 3]
Porcentagem de acertos: 79.27%
Atributos selecionados[2, 3, 4]
Porcentagem de acertos: 79.4%
Atributos selecionados[2, 3, 4, 6]
Porcentagem de acertos: 79.13%
